# This file move the images from the Faces folder to the EiLA_data folder based on the emotion label

This files also split the data into training, validation and testing sets for fine-tuning the models.

In [1]:
import os
import sys
import pandas as pd
import shutil

In [2]:
class7_names = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger']

In [3]:
# read csv file with 2 columns file_name and label and the first row is the header  
def read_csv(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        sys.exit(1)

In [4]:
#the first row in the csv file is the header
mainData = read_csv('../MainDataSaved.csv')

File not found: ../MainDataSaved.csv


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [5]:
# Remove the prefix from the file_name column
mainData['file_name'] = mainData['file_name'].apply(lambda x: x.split('/')[-1])

NameError: name 'mainData' is not defined

In [6]:
# check the folder in face and add another column to the csv file which number of images in each folder which is the file_name
for index, row in mainData.iterrows():
    file_name = row['file_name'].split('.')[0]
    folder_path = os.path.join('../Faces', file_name)
    try:
        images_path = os.listdir(folder_path)
        mainData.at[index, 'num_images'] = len(images_path)
    except FileNotFoundError:
        print(f"Directory not found: {folder_path}. Skipping this file.")

NameError: name 'mainData' is not defined

In [7]:
# using the csv data move each data to the corresponding folder
for i in range(7):
    #filter main data by label
    data = mainData[mainData['label'] == class7_names[i]]
    # print(data)
    for index, row in data.iterrows():
        # print(row)

        # Get file name from the DataFrame
        file_name = row['file_name'].split('.')[0]

        # Create path to the folder
        # images_path = os.listdir(os.path.join('./faces', file_name))
        folder_path = os.path.join('./Faces', file_name)

        # Ensure the target folder exists
        target_folder = os.path.join('../EiLA_data', str(i))
        os.makedirs(target_folder, exist_ok=True)

        # Move each image in the folder to the target folder
        try:
            # Try to list files in the folder
            images_path = os.listdir(folder_path)
            print(images_path)

            # Move each image in the folder to the target folder
            for img in images_path:
                if(img.split('.')[-1] == 'jpg' or img.split('.')[-1] == 'png'):
                    final_image_name = file_name+"_"+img
                    print(os.path.join(target_folder,final_image_name))
                    shutil.copy(os.path.join(folder_path, img), os.path.join(target_folder,final_image_name))

        except FileNotFoundError:
            # If the directory or file doesn't exist, ignore the error and continue
            print(f"Directory not found: {folder_path}. Skipping this file.")

NameError: name 'mainData' is not defined

In [ ]:
for i in range(7):
    path = os.path.join('../EiLA_data', str(i))
    print(f"Number of images in class {i}: {len(os.listdir(path))}")

# Create training, Validation and Testing folders from the above data

In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define folder paths for each split
train_folder = '../split/train'
valid_folder = '../split/valid'
test_folder = '../split/test'
save_data_folder = '../split'

# Create folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(valid_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

def get_file_name(file_name):
    # Split the file name based on underscores and extract the video name
    parts = file_name.split('_')
    video_name = '_'.join(parts[:-3])  # Video name consists of everything before the last three parts
    return video_name

# Assuming 'mainData' is the DataFrame containing the data with columns 'label' and 'file_name'
for i in range(7):
    # Filter main data by label
    data = mainData[mainData['label'] == class7_names[i]]

    # Apply get_file_name to extract the video name
    data['video_name'] = data['file_name'].apply(get_file_name)

    # Get unique video names
    unique_videos = data['video_name'].unique()

    # Split unique videos into train (70%), valid (20%), and test (10%)
    train_videos, temp_videos = train_test_split(unique_videos, test_size=0.3, random_state=42)
    valid_videos, test_videos = train_test_split(temp_videos, test_size=0.3333, random_state=42)  # 0.3333 * 0.3 ≈ 0.1

    # Filter the data by video names to create train, valid, and test subsets
    train_data = data[data['video_name'].isin(train_videos)]
    valid_data = data[data['video_name'].isin(valid_videos)]
    test_data = data[data['video_name'].isin(test_videos)]

    # Function to copy files to target folder
    def copy_files(data_subset, target_folder):
        for index, row in data_subset.iterrows():
            file_name = row['file_name']
            folder_path = os.path.join('../Faces', file_name)
            target_label_folder = os.path.join(target_folder, str(i))
            os.makedirs(target_label_folder, exist_ok=True)

            try:
                images_path = os.listdir(folder_path)
                for img in images_path:
                    if img.split('.')[-1] in ['jpg', 'png']:
                        final_image_name = f"{file_name}_{img}"
                        shutil.copy(os.path.join(folder_path, img), os.path.join(target_label_folder, final_image_name))
            except FileNotFoundError:
                print(f"Directory not found: {folder_path} Skipping this file.")

    # Copy files to train, valid, and test folders
    copy_files(train_data, train_folder)
    copy_files(valid_data, valid_folder)
    copy_files(test_data, test_folder)


NameError: name 'mainData' is not defined

In [ ]:
# Check the number of images in each split
data_folder = '../EiLA_data/split/train'
for i in range(7):
    print(class7_names[i])
    print(f"Number of images in class {i} in train: {len(os.listdir(os.path.join(data_folder, str(i))))}")
    # print(f"Number of images in class {i} in valid: {len(os.listdir(os.path.join(valid_folder, str(i))))}")
    # print(f"Number of images in class {i} in test: {len(os.listdir(os.path.join(test_folder, str(i))))}")